# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [2]:
install.packages("tokenizers")
install.packages("httr")

library(httr)
library(tokenizers)

tokenize_text <- function(text) {
  tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’


Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)



#### b) Make a function generate keys for ngrams.

In [4]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [5]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
  if (length(tokens) < n) return(new.env(parent = emptyenv()))
  tbl <- new.env(parent = emptyenv())
  for (i in seq_len(length(tokens) - n + 1L)) {
    ngram <- tokens[i:(i + n - 2L)]
    next_word <- tokens[i + n - 1L]
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (next_word %in% names(counts)) {
      counts[[next_word]] <- counts[[next_word]] + 1L
    } else {
      counts[[next_word]] <- 1L
    }
    tbl[[key]] <- counts
  }
  tbl
}

#### d) Function to digest the text.

In [6]:
digest_text <- function(text, n) {
  tokens <- tokenize_text(text)
  build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [7]:
digest_url <- function(url, n) {
  res <- httr::GET(url)
  txt <- httr::content(res, as = "text", encoding = "UTF-8")
  digest_text(txt, n)
}

#### f) Function that gives random start.

In [8]:
random_start <- function(tbl, sep = "\x1f") {
  keys <- ls(envir = tbl, all.names = TRUE)
  if (length(keys) == 0) stop("No n-grams available. Digest text first.")
  picked <- sample(keys, 1)
  strsplit(picked, sep, fixed = TRUE)[[1]]
}

#### g) Function to predict the next word.

In [9]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
  key <- paste(ngram, collapse = sep)
  counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
  if (length(counts) == 0) return(NA_character_)
  sample(names(counts), size = 1, prob = as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [10]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.


In [14]:
library(httr)
library(tokenizers)

set.seed(2025)

# a
grimm_url <- "https://www.gutenberg.org/files/2591/2591-0.txt"
grimm_tbl3 <- digest_url(grimm_url, n = 3)
grimm_gen3 <- make_ngram_generator(grimm_tbl3, n = 3)

# (i) Start words = "the king", length = 15
cat("Grimm’s Fairy Tales — start words 'the king':\n")
cat(grimm_gen3(start_words = c("the", "king"), length = 15), "\n\n")

# (ii) No start words, length = 15 (random start)
cat("Grimm’s Fairy Tales — random start:\n")
cat(grimm_gen3(length = 15), "\n")

Grimm’s Fairy Tales — start words 'the king':
the king has forbidden me to marry another husband am not i shall ride upon 

Grimm’s Fairy Tales — random start:
so nimbly and pulled the lace and in short everything that your marriage will soon 


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [17]:
set.seed(2025)
armour_url <- "https://www.gutenberg.org/cache/epub/19447/pg19447.txt"
armour_tbl3 <- digest_url(armour_url, n = 3)
armour_gen3 <- make_ngram_generator(armour_tbl3, n = 3)

# --- (i) Start words = "the king", length = 15 ---
cat("Ancient Armour and Weapons — start words 'the king':\n")
cat(armour_gen3(start_words = c("the", "king"), length = 15), "\n\n")

# --- (ii) No start words, length = 15 ---
cat("Ancient Armour and Weapons — random start:\n")
cat(armour_gen3(length = 15), "\n")

Ancient Armour and Weapons — start words 'the king':
the king in my waking hours no man must be paid to the rose of 

Ancient Armour and Weapons — random start:
three masonic pillars wisdom strength beauty or harmony 826 m triangular plate of the kabalah 


#### c) Explain in 1-2 sentences the difference in content generated from each source.
Grimm's Fairy tale generated content that is in a narrative and story form. The Ancient Armour and Weapons generated content that is more factual indicating it is more informational content compared to the Grimm's story telling contnet. 

## Question 3
#### a) What is a language learning model? 
A language learning model is a probability distribution over words. You provide an input into the probability distribution which conditions the probability distribution, and then you get an output. Essentially, it predicts the next word (token) from the previous words.
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?
To run one locally you can download the pre-trainned model on your computer. Docker and OLLAMA allows you to host the LM locally without needing internet access.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** |Sits between the user and an OS, it shells around the OS. It is a program that lets you interact with all the functionality of a system|
| **Terminal emulator** |The place where the shell sits, it hosts the shell   |
| **Process** | Something that is running on the computer |
| **Signal** |Things we can send to processes to tell them to do something|
| **Standard input** |Each process can read characters from the input|
| **Standard output** |The process can write characters to standard output|
| **Command line argument** |things we pass to a process when we start it  |
| **The environment** |All the stuff a process can see when its running|

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
Find, xargs, and gerp
#### b) Explain what this command is doing, part by part.
1. find . -iname "*.R": Find will search for files and directories, . indicates the current director for find to start searching in, -iname "*.R", will search for files whose names end with .R, and will provide output with a list of .R file paths
2. |: is the pipe operator which will take the command find and pass it as the input to the command xargs grep
3. xargs: Reads the input from find and builds commands from it and apply it to the next command, grep
4. grep read_csv: Searches each file for lines containing the text read_csv, it will print every line from every .R that contains read_csv with the file name

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.

docker run -d -p 8787:8787 -e PASSWORD=yourpassword -v ~/Documents:/home/rstudio rocker/rstudio

Unable to find image 'rocker/rstudio:latest' locally
latest: Pulling from rocker/rstudio
3665120d345d: Pull complete 
b8a35db46e38: Pull complete 
2c9ba66d5dbe: Pull complete 
664fb1818bbb: Pull complete 
abd0190d83fb: Pull complete 
bc9245ceaac5: Pull complete 
5b219f62ce36: Pull complete 
2034506aa72f: Pull complete 
191985778909: Pull complete 
a730ff463d58: Pull complete 
5d246ec925db: Pull complete 
bcce866b1806: Pull complete 
39038e16d1ba: Pull complete 
e2804bef35e8: Pull complete 
cc9c938c1f51: Pull complete 
91ed5b86de88: Pull complete 
08e74fd5985d: Pull complete 
9c1a4a0706b7: Pull complete 
Digest: sha256:9f85211a666fb426081a6f5a01f9f9f51655262258419fa21e0ce38a5afc78d8
Status: Downloaded newer image for rocker/rstudio:latest
586144eb8ff42f918d2f73917a58ddaf099bccf9274e9a7bea1b7d13edeb0b99
#### c) How do you log in to the RStudio server?
http://localhost:8787
Type in username and password 